# ETL script for sodertornsmodellen


Creates ddf--list and ddf--data_for files for Södertornsmodellen and converts shapefiles to GeoJSON.

ARGUMENTS: /path/to/ddf /path/to/raw

SYNOPSIS: python etl--sodertornsmodellen.py /path/to/ddf /path/to/raw

In [21]:
import os
import sys
import zipfile
import pandas as pd
import numpy as np
import re
import shapefile
from json import dumps
reload(sys)
sys.setdefaultencoding("utf-8")

## Functions

In [22]:
def write_ddf_list(df, cols, entity, target_dir, index_file=None, prefix='ddf--list--',value_concepts=None):
    """
    ARGUMENTS
    """  
    outfile = target_dir + prefix + entity + '.csv'
    df[cols].to_csv(outfile, encoding='UTF-8',index=False)
    print 'Printed ' + outfile

In [23]:
def write_ddf_datafor(df, cols, target_dir, dimensions, measure=None, prefix='ddf--data_for--', index_file=None, aliases=None):
    """
    ARGUMENTS
    """
    #assumes measure is given as last element in cols
    if measure is None:
        measure = cols[-1]
    outfile = target_dir + prefix + measure + '--by--' + '--'.join(dimensions) + '.csv'
        
    if aliases is not None:
        df[cols].to_csv(outfile, encoding='UTF-8',index=False, header=aliases)
    else:
        df[cols].to_csv(outfile, encoding='UTF-8',index=False)
    print 'Printed ' + outfile

In [24]:
def shp2geojson(shp_file, json_file=None):
    
    #read shapefile
    reader = shapefile.Reader(shp_file)
    fields = reader.fields[1:]
    field_names = [field[0] for field in fields]
    features = []
    for sr in reader.shapeRecords():
        atr = dict(zip(field_names, sr.record))
        geom = sr.shape.__geo_interface__
        features.append(dict(type='Feature', geometry=geom, properties=atr))

    if json_file is not None:
        #write GeoJSON file
        geojson = open(json_file, 'w')
        geojson.write(dumps({'type': 'FeatureCollection', 'features': features}, indent=2) + "\n")
        geojson.close()
        print 'Printed ' + json_file
    
    return features

In [25]:
def excel2csv(file_xlx, file_csv):
    df = pd.read_excel(file_xlx)
    df.to_csv(file_csv, index=False)
    print 'Printed ' + file_csv
    del df

## Lookup tables used in correcting/translating column names

In [26]:
#replacements using regex
rep = {unicode('å'): 'a',
       unicode('ä'): 'a',
       unicode('ö'): 'o',
       unicode('år'): 'year',
       '  ': '',
       ' ': '_',
       'plomr namn': 'planomradesnamn'
      }
rep = dict((re.escape(k), v) for k, v in rep.iteritems())
pattern = re.compile("|".join(rep.keys()))

#example: the call below will return 'text' with replacements according to 'rep'
#pattern.sub(lambda m: rep[re.escape(m.group(0))], text) 

## Set paths

In [27]:
#take folder separator as per os
pathspt = os.path.sep

#define root path
root = '..' + pathspt + '..' + pathspt + '..' + pathspt + 'ddf--sodertornsmodellen--testing2016'
#root=str(sys.argv[1])

#path to raw data files (cannot get it directly from github since repo is private)
raw = '..' + pathspt + '..' + pathspt + '..' + pathspt + 'raw--sodertornsmodellen--testing2016'
#raw = str(sys.argv[2])

## Load raw data

In [28]:
basomrade_file = raw + pathspt + 'Kopia av Basområden utan småområden 151103.xlsx'
kommun_file = raw + pathspt + 'Kopia av kommuner mindre fil 151022.xlsx'
df_kommun = pd.read_excel(kommun_file,'Blad1')
df_basomrade = pd.read_excel(basomrade_file,'Blad1')

## Pre-processing (remove unnecessary columns and modify column names) 

In [29]:
del df_kommun['Kommun.1']
df_kommun.columns = [pattern.sub(lambda m: rep[re.escape(m.group(0))], col.lower()) for col in df_kommun.columns]
df_basomrade.columns = [pattern.sub(lambda m: rep[re.escape(m.group(0))], col.lower()) for col in df_basomrade.columns]

## ddf--list

In [30]:
geo_dims = ['kommun','basomrade']
time_dims = ['year']
list_cols_kommun = 'kommun'
list_cols_basomrade = ['basomradeskod', 'basomrade','kommun']
list_header_kommun = ['geo', 'name', 'is.kommun']
list_header_basomrade = ['geo','name','kommun','is.basomrade']

#drop duplicates
s_list_kommun_name = df_kommun[list_cols_kommun].drop_duplicates()
df_list_basomrade = df_basomrade[list_cols_basomrade].drop_duplicates(subset = 'basomrade')

#create geo id for kommun by making lowercase and removing åäö and blank space
geo_id = [pattern.sub(lambda m: rep[re.escape(m.group(0))], kommun.lower()) for kommun in s_list_kommun_name]
s_list_kommun_geo = pd.Series(geo_id, name = 'geo')

#concatenate series
df_list_kommun = pd.concat([s_list_kommun_geo, s_list_kommun_name], axis = 1)
del s_list_kommun_name
del s_list_kommun_geo

#create a lookup table that converts name (kommun) to geo (id)
kommun2id = {row.kommun : row.geo for i,row in df_list_kommun.iterrows()}

#add is.kommun and is.basomrade
df_list_kommun['is.kommun'] = True
df_list_basomrade['is.basomrade'] = True

#change headers
df_list_kommun.columns = list_header_kommun
df_list_basomrade.columns = list_header_basomrade

#populate column 'kommun' with geo id instead of name
df_list_basomrade['kommun'] = [kommun2id[unicode(kommun)] for kommun in df_list_basomrade['kommun']]

#add lonlat for kommuner
df_lonlat = pd.read_csv(root + pathspt + 'data_process' + pathspt + 'coords_kommuner.csv')
df_list_kommun['latitude'] = df_lonlat['geo.latitude']
df_list_kommun['longitude'] = df_lonlat['geo.longitude']

#write csv files
frames = [df_list_kommun, df_list_basomrade]
for i,df in enumerate(frames):
    write_ddf_list(df, df.columns, geo_dims[i], root + pathspt, prefix='ddf--list--geo--')

#free space
del df_lonlat
del df_list_kommun
del df_list_basomrade

# ddf--data_for

In [31]:
measures = ['medelinkomst', 'andel_hogskoleutbildade', 'befolkning', 'forvarvsfrekvens', 'flytt_arbete', 'flytt_utbildning']
datafor_kommun_fixedcols = ['geo', 'kommun', 'year']
datafor_basomrade_fixedcols = ['basomradeskod', 'basomrade', 'year']
fixedcols = [datafor_kommun_fixedcols, datafor_basomrade_fixedcols]
kommun_dims = ['kommun', 'year']
basomrade_dims = ['basomrade', 'year']
dims = [kommun_dims, basomrade_dims]

#sort tables by kommun/basomrade and year
df_kommun_sorted = df_kommun.sort_values(kommun_dims)
df_basomrade_sorted = df_basomrade.sort_values(basomrade_dims)

#insert a geo (id) column
df_kommun_sorted.insert(0, 'geo', [kommun2id[unicode(kommun)] for kommun in df_kommun_sorted['kommun']])

#write csv files
frames = [df_kommun_sorted, df_basomrade_sorted]
for i, df in enumerate(frames):
    for measure in measures:
        write_ddf_datafor(df, fixedcols[i] + [measure], root + pathspt, dims[i], aliases=['geo', 'name', 'year', measure])    

#free space
del df_kommun
del df_basomrade
del df_kommun_sorted
del df_basomrade_sorted

## ddf--measures

In [32]:
#create xlsx for ddf--measures (this can later be edited manually)
measuresfile_xlx = root + pathspt + 'data_process' + pathspt + 'ddf--measures.xlsx'
measuresfile_csv = root + pathspt + 'ddf--measures.csv'

if not os.path.isfile(measuresfile_xlx):
    measures_header = ['measure', 'name', 'name_short', 'description']
    df_measures = pd.DataFrame(columns = measures_header)
    df_measures['measure'] = measures
    #output as excel sheet
    df_measures.to_excel(measuresfile_xlx, index=False)
    del df_measures
    print 'Printed' + measuresfile_xlx
    excel2csv(measuresfile_xlx, measuresfile_csv)
else:
    print 'File: ' + measuresfile_xlx + ' already exists.'

## ddf--dimensions

In [33]:
#use ddf--dimension from systema globalis as template
dimfile_xlx = root + pathspt + 'data_process' + pathspt + 'ddf--dimensions.xlsx'
dimfile_csv = root + pathspt + 'ddf--dimensions.csv'
csv_url = 'https://raw.githubusercontent.com/valor-software/ddf--gapminder--systema_globalis/master/ddf--dimensions.csv'

if not os.path.isfile(dimfile_xlx):
    df_dim = pd.read_csv(csv_url)
    #delete last column (it's empty)
    del df_dim[df_dim.columns[-1]]
    #find rows with type = dimension or name = Year and drop the rest
    df_dim_reduced = df_dim[(df_dim['type'] == 'dimension') | (df_dim['name'] == 'Year')]
    #add map column
    df_dim_reduced['map'] = ''
    df_dim_reduced.to_excel(dimfile_xlx, index=False)
    del df_dim
    print 'Printed ' + dimfile_xlx
    excel2csv(dimfile_xlx, dimfile_csv)
else:
    print 'File: ' + dimfile_xlx + ' already exists.'  

## Extract zips, convert shapefiles to geojson and write to file

In [34]:
extract_dir = root + pathspt + 'extracted'
json_dir = root + pathspt + 'geojson'
input_names = ['Bas_Sodertorn_2000', 'Bas_Sodertorn_2010']
zip_files = [raw + pathspt + name + '.zip' for name in input_names] 

if not os.path.isdir(extract_dir):
    os.makedirs(extract_dir)
    
if not os.path.isdir(json_dir):
    os.makedirs(json_dir)
    
for idx, zfile in enumerate(zip_files):
    fh = open(zfile, 'rb')
    z = zipfile.ZipFile(fh)

    #extract files
    for name in z.namelist():
        z.extract(name, extract_dir)
        if name[-4:] == '.shp':
            shp_file = name
            
    #convert shapefile to geojson
    shp2geojson(extract_dir + pathspt + shp_file, json_dir + pathspt + input_names[idx].lower() + '.json')
    
    fh.close()
    z.close()

## ddf--list--geo--basomrade--map

In [35]:
shpfile = root + pathspt + 'extracted' + pathspt + 'Bas_Sodertorn_2010' + pathspt + 'Basomr_Sod_2010.shp'
features = shp2geojson(shpfile)
listfile = root + pathspt + 'ddf--list--geo--basomrade.csv'
mapfile = root + pathspt + 'ddf--list--geo--basomrade--map.csv'
shpfile_identifier = 'BASKOD2010'

#load ddf--list--geo--basomrade and add column map
df_list = pd.read_csv(listfile)
df_geoshape = df_list['geo'].to_frame()
df_geoshape['map'] = ''

#add geojson feature to each basomrade
for feature in features:
    baskod = feature['properties'][shpfile_identifier]
    rowindex = df_geoshape.loc[df_list['geo'] == baskod].index
    df_geoshape.loc[rowindex, 'map'] = str(feature)

#write to csv
df_geoshape.to_csv(mapfile, index=False)
print 'Printed ' + mapfile

#free space
del df_list
del df_geoshape

## ddf--index

In [36]:
index_header = ['file', 'value_concept', 'geo', 'time']
outfile = root + pathspt + 'ddf--index.csv'

#specify value_concepts of list files (cannot read this based on file name only as we do for data_for)
#would also be possible to read in csv in order to get the same info but that way is much slower
lookup_value_concept = {'kommun': ['name', 'is.kommun','latitude','longitude'], 'basomrade': ['name','kommun','is.basomrade'], 'map': ['map']}

#create ddf--index file
pd.DataFrame(columns = index_header).to_csv(outfile, encoding='UTF-8', index=False)
indexfile = open(outfile, 'a')

for item in os.listdir(root + pathspt):
    #add enties for data_for files
    if re.search('--data_for--', item):
        fragments = item.split('.')[0].split('--')
        time = fragments[-1]
        geo = fragments[-2]
        value_concept = fragments[-4]
        indexfile.write(','.join([item, value_concept, geo, time]) + '\n')
    
    #add entries for ddf--list files (for now this assumes that the only list--files that exist are list--geo)   
    elif re.search('--list--geo', item):
        fragments = item.split('.')[0].split('--')
        if fragments[-1] == 'map':
            geo = fragments[-2] 
        else:
            geo = fragments[-1]
        time = ''
        value_concepts = lookup_value_concept[fragments[-1]]
        for vc in value_concepts:
            indexfile.write(','.join([item, vc, geo, time]) + '\n')
            
print 'Printed ddf--index.csv' 
indexfile.close()